# Pracownia z Analizy Numerycznej
## Zadanie 3.20
## Wyznaczanie macierzy odwrotnej za pomocą rozkładów QR oraz LU
# Autorzy:
## Artur Derechowski
## Mateusz Markiewicz

# Rozkład LU

Rozkład LU macierzy A

In [1]:
function LU_decomposition(A)
    n = size(A)[1]
    L = zeros(Float64,n,n)
    U = zeros(Float64,n,n)
    function sum_of_mults(A,B,i,j)
        s = Float64(0)
        for k = 1:i-1
            s += A[i,k] * B[k,j]
        end
        return s
    end
    function sum_of_mults2(A,B,i,j)
        s = Float64(0)
        for k = 1:i-1
            s += A[j,k] * B[k,i]
        end
        return s
    end
    for i = 1:n
        U[i,i] = A[i,i] - sum_of_mults(L,U,i,i)
        L[i,i] = 1
        for j = (i+1):n
            U[i,j] = A[i,j] - sum_of_mults(L,U,i,j)
            L[j,i] = 1/U[i,i] * (A[j,i] - sum_of_mults2(L,U,i,j))
        end
    end
    return L,U
end
    

LU_decomposition (generic function with 1 method)

Przykład działania

In [2]:
A = [[1 4 7];[2 5 8];[3 6 9]]

3×3 Array{Int64,2}:
 1  4  7
 2  5  8
 3  6  9

In [3]:
L,U = LU_decomposition(A)

([1.0 0.0 0.0; 2.0 1.0 0.0; 3.0 2.0 1.0], [1.0 4.0 7.0; 0.0 -3.0 -6.0; 0.0 0.0 0.0])

In [4]:
L

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 2.0  1.0  0.0
 3.0  2.0  1.0

In [5]:
U

3×3 Array{Float64,2}:
 1.0   4.0   7.0
 0.0  -3.0  -6.0
 0.0   0.0   0.0

In [6]:
L*U

3×3 Array{Float64,2}:
 1.0  4.0  7.0
 2.0  5.0  8.0
 3.0  6.0  9.0

Funkcja do rozwiązywania układów równań A \* X = B za pomocą metory LU L\*(U\*X) = B

In [7]:
function solve(L,U,B)
    n = size(L)[1]
    Y = zeros(Float64,n,1)
    X = zeros(Float64,n,1)
    Y[1] = B[1]
    function sum(L,Y,i)
        s = Float64(0)
        for j = 1:i-1
            s += L[i,j] * Y[j]
        end
        return s
    end
    for i = 2:n
        Y[i] = B[i] - sum(L,Y,i)
    end
    function sum2(U,X,i,n)
        s = Float64(0)
        for j = (i+1):n
            s += U[i,j] * X[j]
        end
        return s
    end
    X[n] = Y[n] / U[n,n]
    for i = (n-1):-1:1
        X[i] = 1/U[i,i] * (Y[i] - sum2(U,X,i,n))
    end
    return X
end 

solve (generic function with 1 method)

Przykład działania

In [8]:
A2 = [[4 -2 4 -2];[3 1 4 2];[2 4 2 1];[2 -2 4 2]]

4×4 Array{Int64,2}:
 4  -2  4  -2
 3   1  4   2
 2   4  2   1
 2  -2  4   2

In [9]:
b = [8; 7; 10; 2]

4-element Array{Int64,1}:
  8
  7
 10
  2

In [10]:
L2,U2 = LU_decomposition(A2)

([1.0 0.0 0.0 0.0; 0.75 1.0 0.0 0.0; 0.5 2.0 1.0 0.0; 0.5 -0.4 -1.2 1.0], [4.0 -2.0 4.0 -2.0; 0.0 2.5 1.0 3.5; 0.0 0.0 -2.0 -5.0; 0.0 0.0 0.0 -1.6])

In [11]:
solve(L2,U2,b)

4×1 Array{Float64,2}:
 -1.0000000000000009
  2.0000000000000004
  3.000000000000001 
 -2.0000000000000004

# Wyznaczanie A^-1 za pomocą rozkładu LU

Niech X = A^-1

In [12]:
function invertible(A)
    n = size(A)[1]
    L,U = LU_decomposition(A)
    B = zeros(Float64,1,n)
    B[1] = 1
    X = solve(L,U,B)
    for i = 2:n
        B = zeros(Float64,n)
        B[i] = 1
        X = [X (solve(L,U,B))]
    end
    return X
end
    

invertible (generic function with 1 method)

Przykład działania

In [13]:
A = [[2 -2 18];[2 1 0];[1 2 0]]

3×3 Array{Int64,2}:
 2  -2  18
 2   1   0
 1   2   0

In [14]:
A * invertible(A)

3×3 Array{Float64,2}:
 1.0  0.0           0.0        
 0.0  1.0          -1.11022e-16
 0.0  1.11022e-16   1.0        

# Rozkład QR

In [15]:
using LinearAlgebra

Druga norma wektorowa

In [16]:
function _norm(v)
    n = 0.0
    for i = 1:length(v)
        n += v[i]*v[i]
    end
    return sqrt(n)
end
#norm([1,2,3]) #powinno 14

_norm (generic function with 1 method)

Metoda householdera

In [17]:
function householder(A, k) #k jest którą macierz Householdera liczymy
    n = size(A)[1]
    v = zeros(Float64,(n-k) +1)
    for i = k:n
        v[i-k+1] = A[i,k]
    end
    v[1] = (Float64(sign(A[k,k]))*norm(v)) + v[1]
    vt = transpose(v)
    H = -2 * v*vt / (vt*v) + I
    ret = zeros(n,n)
    for i = 1:k-1
        ret[i,i] = 1
    end
    for i = k:n
        for j = k:n
            ret[i,j] = H[i-k+1,j-k+1]
        end
    end
    return ret
end

#############################TEST
B =  [1 2 3.0;
      4 5 6;
      7 8 9]
#  H1 = householder(B,1)
#  B = H1*B

# H1 = householder(A, 1)
# A = H1*A
# H2 = householder(A, 2)
# A = H2*A
#H3 = householder(H2, 3)

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

# Rozkład QR

In [18]:
function QRfactorize(A) #rozkład na macierze Q,R takie, że QR=A
    n = size(A)[1]
    Q = I
    H = A
    for i = 1:n
        H = householder(A, i)
        A = H * A
        Q = Q*H
    end
    return Q,A
end   
############################TEST
B =  [1 2 3.0;
      4 5 6;
      7 8 9]
_B = QRfactorize(B)
_B[1] * _B[2]


3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

Odwracanie macierzy trójkątnej

In [19]:
function inv_tri(B) #odwrotność macierzy górnotrójkątnej B w O(n^3)
    n = size(B)[1]
    A = deepcopy(B)
    for i = 1:n
        A[i,i] = 1.0/A[i,i]
    end
    for i = n-1:-1:1
        for j = n:-1:i+1
            s = 0.0
            for k = i+1:j
                s += A[i,k]*A[k,j]
            end
            A[i,j] = - A[i,i]*s
        end
    end
    return A
end
########################TEST
B =  [1 2 3;
      0 5 6;
      0 0 9.0]
inv_tri(inv_tri(B))

#  1.0  -0.4  -0.0666667
#  0.0   0.2  -0.133333 
#  0.0   0.0   0.111111

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 0.0  5.0  6.0
 0.0  0.0  9.0

# Odwracanie macierzy metodą QR

In [63]:
function QRinv(A)
    Q,R = QRfactorize(A)
    R_i = inv(UpperTriangular(R))
    Q_t = transpose(Q)
    return R_i*Q_t
end

B = [2 -2 18;
     7 124 9;
     9 2 2]

#######################################TEST
QRinv(QRinv(B))


3×3 Array{Float64,2}:
 2.0   -2.0  18.0
 7.0  124.0   9.0
 9.0    2.0   2.0

# Porównanie obu metod

Pierwsza norma macierzowa

In [21]:
function first_norm(A)
    n = size(A)[1]
    m = 0
    for i = 1:n
        m = max(m, sum(map(abs,A[:,i])))
    end
    return m
end

first_norm (generic function with 1 method)

In [22]:
A10 = [[4 -2 4 -2];[3 1 4 2];[2 4 2 1];[2 -2 4 2]]

4×4 Array{Int64,2}:
 4  -2  4  -2
 3   1  4   2
 2   4  2   1
 2  -2  4   2

In [23]:
first_norm(A10)

14

Funkcję do testowania rozkładów QR i LU

In [24]:
function test_LU(A)
    return abs(1 - first_norm(A * invertible(A)))
end

function test_QR(A)
    return abs(1 - first_norm(A*QRinv(A)))
end

test_QR (generic function with 1 method)

# TESTY

In [46]:
A81 = rand(-10:10,15,15)
A8 = zeros(Float64,15,15)
for i = 1:15
    for j = 1:15
        A8[i,j] = A81[i,j]
    end
    A8[i,i] = A8[i,i] * 10000000000000 + rand(1:10)
end
A8

15×15 Array{Float64,2}:
   6.0e13    1.0      3.0      5.0     …    1.0      5.0      4.0      0.0   
  10.0      -2.0e13   2.0      8.0          4.0     -2.0      7.0     -9.0   
  -5.0       8.0      3.0e13   2.0         -4.0     -1.0      4.0      1.0   
  -8.0      -8.0      0.0      3.0e13      -5.0      5.0      8.0     -1.0   
   4.0      10.0      4.0     -9.0         -1.0      1.0      8.0     -8.0   
   6.0      -2.0     -5.0     10.0     …   -7.0     -9.0      9.0      9.0   
  -1.0      -1.0     -9.0      4.0         -8.0      7.0     -3.0     -2.0   
 -10.0      -9.0      0.0     -5.0        -10.0     -9.0     -1.0      1.0   
   5.0     -10.0     -2.0      9.0         -3.0     10.0     -6.0      8.0   
   6.0     -10.0      1.0      9.0          7.0     -8.0     -9.0      3.0   
   2.0       1.0     -8.0     -1.0     …    3.0     -6.0     -3.0      0.0   
  -9.0      -9.0      8.0     -7.0          3.0e13   7.0      3.0     -9.0   
  -4.0      10.0      0.0     -4.0      

In [47]:
det(A8)

5.4867456000027836e190

Dokładność metody LU

In [48]:
test_LU(A8)

1.1102230246251565e-15

Dokładność metody QR

In [49]:
test_QR(A8)

1.1102230246251565e-15

Dokładność rozkładu QR

In [50]:
Q8,R8 = QRfactorize(A8)
first_norm(Q8 * R8 - A8)

0.02401524245059461

Dokładność rozkładu LU

In [51]:
L8,U8 = LU_decomposition(A8)
first_norm(L8*U8 - A8)

4.440892098500667e-15

Dokładność funkcji wbudowanej do rozkładu QR

In [52]:
F = qr(A8)
first_norm(F.Q * F.R - A8)

0.010864257812528422

# Badanie dokładności dla macierzy A z coraz mocnej dominującą przekątną

In [32]:
B1 = zeros(Float64,100,100)
B11 = rand(-50:50,100,100)
for i = 1:100
    for j = 1:100
        B1[i,j] = B11[i,j]
    end
end
B1

100×100 Array{Float64,2}:
 -16.0  -36.0   44.0    4.0  -11.0  …  -13.0   40.0  -20.0   30.0   39.0
  50.0    8.0   -6.0   -1.0  -45.0     -44.0   -6.0   -1.0  -21.0    8.0
   0.0   27.0  -24.0    5.0   46.0      -1.0   12.0    9.0   -7.0    6.0
 -37.0    5.0   26.0  -11.0   -4.0      32.0   47.0    1.0  -23.0  -23.0
 -45.0   -2.0   12.0   33.0   28.0       2.0   32.0   -1.0   32.0    1.0
  12.0    4.0   41.0   50.0    2.0  …  -23.0   27.0  -27.0  -10.0  -45.0
 -40.0   32.0  -49.0   19.0   42.0     -12.0    3.0  -12.0  -23.0  -15.0
  40.0  -31.0  -16.0  -36.0   -8.0       7.0   16.0  -32.0  -15.0  -46.0
 -39.0  -49.0   -6.0  -45.0   11.0      12.0   41.0   38.0   13.0   42.0
  29.0  -43.0   27.0  -23.0   44.0     -37.0  -29.0   36.0  -16.0  -33.0
 -22.0  -26.0    6.0  -49.0   24.0  …   40.0   36.0  -45.0   26.0   31.0
   9.0   27.0  -34.0  -50.0   29.0      49.0   -5.0   24.0  -47.0   49.0
  17.0    7.0   -9.0  -31.0    3.0      43.0  -48.0  -45.0   14.0  -10.0
   ⋮                     

In [33]:
function test(A,n)
    cA = copy(A)
    err_LU = zeros(n)
    err_QR = zeros(n)
    m = 10000000/n
    for k = 1:n
        for i = 1:100
            cA[i,i] += rand() * m
        end
        err_LU[k] = test_LU(cA)
        err_QR[k] = test_QR(cA)
    end
    return err_LU,err_QR
end

test (generic function with 1 method)

In [34]:
err_LU,err_QR = test(B1,100)

([1.11022e-15, 1.33227e-15, 8.88178e-16, 1.11022e-15, 8.88178e-16, 8.88178e-16, 1.33227e-15, 1.55431e-15, 1.11022e-15, 8.88178e-16  …  1.33227e-15, 1.33227e-15, 6.66134e-16, 1.33227e-15, 8.88178e-16, 6.66134e-16, 1.33227e-15, 1.11022e-15, 8.88178e-16, 1.33227e-15], [6.21725e-15, 9.32587e-15, 6.21725e-15, 5.9952e-15, 6.88338e-15, 7.32747e-15, 5.32907e-15, 6.43929e-15, 8.21565e-15, 6.66134e-15  …  4.88498e-15, 6.88338e-15, 5.10703e-15, 5.10703e-15, 5.9952e-15, 5.32907e-15, 6.88338e-15, 5.9952e-15, 4.88498e-15, 5.32907e-15])

In [62]:
using Plots
plotly()
plot(1:100,err_LU,seriestype=:scatter,label = "Metoda LU", title = "Dokładność wyznaczania macierzy odwrotnej",size = (800, 400),xlabel = "n",ylabel = "Błąd wyniku")
plot!(1:100,err_QR,seriestype=:scatter,label = "Metoda QR")

# Badanie dokładności dlamacierzy A postaci Q^T * B * Q

In [36]:
QB1,RB1 = QRfactorize(B1)
B2 = zeros(Float64,100,100)
l = rand(0:100)
e = 0
for i = 100:-1:1
    e += rand()/1000
    B2[i,i] = l + e
end
B2

100×100 Array{Float64,2}:
 86.0441   0.0     0.0      0.0     …   0.0     0.0      0.0      0.0   
  0.0     86.044   0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0    86.0436   0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0     86.0435      0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0     …   0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0     …   0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  0.0      0.0     0.0      0.0         0.0     0.0      0.0      0.0   
  ⋮                      

In [37]:
function test2(A,n)
    cA = copy(A)
    err_LU = zeros(n)
    err_QR = zeros(n)
    for k = 1:n
        l = rand(10:100)
        for i = 100:-1:1
            l += rand(1:10)
            cA[i,i] += l
        end
        err_LU[k] = test_LU((transpose(QB1)*cA*QB1))
        err_QR[k] = test_QR((transpose(QB1)*cA*QB1))
    end
    return err_LU,err_QR
end

test2 (generic function with 1 method)

In [38]:
err_LU2,err_QR2 = test2(B2,100)

([2.66454e-15, 3.9968e-15, 4.21885e-15, 5.10703e-15, 3.33067e-15, 3.55271e-15, 3.77476e-15, 3.77476e-15, 4.21885e-15, 4.44089e-15  …  3.9968e-15, 5.10703e-15, 4.66294e-15, 3.9968e-15, 4.44089e-15, 4.21885e-15, 4.44089e-15, 4.21885e-15, 3.77476e-15, 4.21885e-15], [2.39808e-14, 2.4647e-14, 2.33147e-14, 3.04201e-14, 2.66454e-14, 2.62013e-14, 3.37508e-14, 2.73115e-14, 2.59792e-14, 2.66454e-14  …  2.62013e-14, 3.26406e-14, 2.57572e-14, 2.81997e-14, 3.15303e-14, 2.84217e-14, 2.9976e-14, 2.68674e-14, 2.62013e-14, 2.93099e-14])

In [39]:
plot(1:100,err_LU2,seriestype=:scatter,label = "Metoda LU", title = "Dokładność wyznaczania macierzy odwrotnej",size = (800, 400),xlabel = "n",ylabel = "Błąd wyniku")
plot!(1:100,err_QR2,seriestype=:scatter,label = "Metoda QR")

# Badanie dokładności dla macierzy A z losowymi wartościami

In [40]:
function test3(n,size,max)
    err_LU = zeros(Float64,n)
    err_QR = zeros(Float64,n) 
    for k = 1:n
        cA = zeros(Float64,size,size)
        rando = rand(-max:max,size,size)
        for i = 1:size
            for j = 1:size
                cA[i,j] = rando[i,j]
            end
        end
        err_LU[k] = test_LU(cA)
        err_QR[k] = test_QR(cA)
    end
    return err_LU,err_QR
end

test3 (generic function with 1 method)

In [55]:
err_LU3,err_QR3 = test3(100,50,100)

([1.28764e-12, 8.54428e-12, 5.99092e-11, 5.42189e-12, 4.63607e-12, 7.01375e-11, 1.70731e-10, 1.17217e-12, 9.77218e-13, 1.74283e-12  …  4.55613e-12, 1.5268e-11, 1.75771e-12, 2.72338e-12, 5.71676e-12, 3.40861e-12, 1.06313e-11, 4.35962e-12, 3.08227e-11, 2.13762e-12], [9.37028e-14, 1.76303e-13, 1.40998e-13, 1.83409e-13, 2.23821e-13, 1.69642e-13, 6.46372e-13, 9.83658e-14, 1.25899e-13, 4.26326e-14  …  2.34701e-13, 1.75193e-13, 3.57492e-14, 1.52323e-13, 1.35003e-13, 8.08242e-14, 8.70415e-14, 9.41469e-14, 6.52589e-13, 7.66054e-14])

In [56]:
plot(1:100,err_LU3,seriestype=:scatter,label = "Metoda LU", title = "Dokładność wyznaczania macierzy odwrotnej",size = (800, 400),xlabel = "n",ylabel = "Błąd wyniku")
plot!(1:100,err_QR3,seriestype=:scatter,label = "Metoda QR")

# Badanie numeryczenej poprawności obu metod

In [59]:
err_2 = zeros(Float64,100)
err_3 = zeros(Float64,100)
for k = 1:100
    B3 = zeros(Float64,25,25)
    B3f = zeros(Float64,25,25)
    B31 = rand(-50:50,25,25)
    for i = 1:25
        for j = 1:25
            B3[i,j] = B31[i,j]
            B3f[i,j] = B31[i,j] + rand()/10000000000000
        end
    end
    err_2[k] = first_norm(QRinv(B3)-QRinv(B3f))
    err_3[k] = first_norm(invertible(B3)-invertible(B3f))
end

In [60]:
plot(1:100,err_2,ylim = (0,5*10^-13),seriestype=:scatter, title = "Poprawność numeryczna metody QR",size = (800, 400),xlabel = "n",ylabel = "Błąd wyniku")


In [61]:
plot(1:100,err_3,seriestype=:scatter, title = "Poprawność numeryczna metody LU",size = (800, 400),xlabel = "n",ylabel = "Błąd wyniku")